In [ ]:
#pip install split-folders 

In [ ]:
import os
import numpy as np
import tifffile as tiff
#import cv2
from PIL import Image
from patchify import patchify
from matplotlib import pyplot as plt
import random
import splitfolders  # or import split_folders
import glob
import rasterio
import geopandas as gpd

In [ ]:
os.getcwd()
root_directory = 'big_tree'
os.chdir(root_directory)

In [ ]:
patch_size = 128
image_dir = os.path.join("predictors/")
mask_dir = os.path.join(root_directory, "reference/Masks/")
os.makedirs("128_patches_NN", exist_ok=True)
os.makedirs("128_patches_NN/train/", exist_ok=True)
os.makedirs("128_patches_NN/train/images", exist_ok=True)
os.makedirs("128_patches_NN/train/masks", exist_ok=True)
os.makedirs("128_patches_NN/val/", exist_ok=True)
os.makedirs("128_patches_NN/val/images", exist_ok=True)
os.makedirs("128_patches_NN/val/masks", exist_ok=True)
os.makedirs("128_patches_NN/test/", exist_ok=True)
os.makedirs("128_patches_NN/test/images", exist_ok=True)
os.makedirs("128_patches_NN/test/masks", exist_ok=True)

out_img_train = os.path.join(root_directory, "128_patches_NN/train/images")
out_mask_train = os.path.join(root_directory, "128_patches_NN/train/masks")
out_img_val = os.path.join(root_directory, "128_patches_NN/val/images")
out_mask_val = os.path.join(root_directory, "128_patches_NN/val/masks")
out_img_test = os.path.join(root_directory, "128_patches_NN/test/images")
out_mask_test = os.path.join(root_directory, "128_patches_NN/test/masks")

In [ ]:
# Load the shapefile
shapefile_path = os.path.join('tiles.shp')
gdf = gpd.read_file(shapefile_path)

# Print all column names
print(gdf.columns)



In [ ]:
# Extract the values of a specific column as a list
column_name = 'Tile'  # Replace with your actual column name
column_values = gdf[column_name].tolist()

# Print the extracted list of values
print(column_values)

In [ ]:
numvar = [None]*len(column_values)

for i in range(len(column_values)):
    numvar[i]="{:03d}".format(column_values[i])
    
print(numvar)

In [ ]:
# Your original list
numvar = numvar

# Set a seed for reproducibility
random.seed(42)

# Shuffle the list to ensure randomness
random.shuffle(numvar)

# Get the number of elements in each split
train_size = int(0.75 * len(numvar))
val_size = int(0.15 * len(numvar))

# Split the list based on the calculated sizes
train = numvar[:train_size]
val = numvar[train_size:train_size + val_size]
test = numvar[train_size + val_size:]

# Print the results
print("Train set:", train)
print("Validation set:", val)
print("Test set:", test)

In [ ]:
for i in Train_set:
    num = "{:01d}".format(int(i)) 
    print(num)
print(gdf['Tile'].tolist())
print(gdf['Tile'].dtype)  # Should be 'int64' or 'int32'
print(type(Train_set[0]))  # Should be <class 'int'>
print(Train_set)
print(Val_set)
print(Test_set)
print(gdf['Tile'].tolist())  # Compare this with the sets

In [ ]:
# Convert sets from strings to integers
Train_set = [int(x) for x in Train_set]
Val_set = [int(x) for x in Val_set]
Test_set = [int(x) for x in Test_set]

gdf['data_split'] = None  # Initialize the entire column with None values

# Assign 'Train', 'Validation', and 'Test'
for i in Train_set:
    gdf.loc[gdf['Tile'] == i, 'data_split'] = 'Train'

for i in Val_set:
    gdf.loc[gdf['Tile'] == i, 'data_split'] = 'Validation'

for i in Test_set:
    gdf.loc[gdf['Tile'] == i, 'data_split'] = 'Test'

# Check the updated DataFrame
print(gdf[['Tile', 'data_split']])

In [ ]:
# Define edge colors for each class
edge_colors = {
    'Test': 'red',
    'Validation': 'blue',
    'Train': 'green'#,

    # None: 'gray'  # Optional, for unassigned data_split rows
}

# Create a color column in gdf based on data_split
gdf['edgecolor'] = gdf['data_split'].map(edge_colors)

# Plotting the geodataframe with edge colors for each data_split class (contour only)
fig, ax = plt.subplots(figsize=(10, 8))
gdf.plot(ax=ax, edgecolor=gdf['edgecolor'], facecolor='none', linewidth=1.5)

# Add a legend manually
import matplotlib.patches as mpatches
legend_patches = [mpatches.Patch(color=color, label=label) for label, color in edge_colors.items()]
plt.legend(handles=legend_patches, title="Data Split")

# Display the plot
plt.title("Contours of gdf by data_split")
plt.show()


In [ ]:
# Save as a Shapefile
gdf.to_file(os.path.join(root_directory,"data_split.shp"), driver='ESRI Shapefile')

In [ ]:
# Define the pattern to match tile files (e.g., 'tile_*.tif')
tile_pattern = os.path.join(image_dir, 'Sentinel_*.tif')

# Use glob to find all tile file paths matching the pattern
tile_paths = glob.glob(tile_pattern)

# Filter paths to include only the desired numbers
Train_image_paths = [tile_path for tile_path in tile_paths if any(num in tile_path for num in train)]
Val_image_paths = [tile_path for tile_path in tile_paths if any(num in tile_path for num in val)]
Test_image_paths = [tile_path for tile_path in tile_paths if any(num in tile_path for num in test)]

# Print the list of filtered tile file paths
print("Filtered test file paths:")
for tile_path in Test_image_paths:
    print(tile_path)
print("Filtered val file paths:")
for tile_path in Val_image_paths:
    print(tile_path)
print("Filtered train file paths:")
for tile_path in Train_image_paths:
    print(tile_path)

In [ ]:
# Define the pattern to match tile files (e.g., 'tile_*.tif')
mask_pattern = os.path.join(mask_dir, 'Mask_*.tif')

# Use glob to find all tile file paths matching the pattern
mask_paths = glob.glob(mask_pattern)

# Filter paths to include only the desired numbers
Train_mask_paths = [tile_path for tile_path in mask_paths if any(num in tile_path for num in train)]
Val_mask_paths = [tile_path for tile_path in mask_paths if any(num in tile_path for num in val)]
Test_mask_paths = [tile_path for tile_path in mask_paths if any(num in tile_path for num in test)]

# Print the list of filtered tile file paths
print("Filtered test file paths:")
for tile_path in Test_mask_paths:
    print(tile_path)
print("Filtered val file paths:")
for tile_path in Val_mask_paths:
    print(tile_path)
print("Filtered train file paths:")
for tile_path in Train_mask_paths:
    print(tile_path)

In [ ]:
img = tiff.imread(Train_image_paths[1])
print(img.shape)
img = np.transpose(img, (1, 2, 0))
print(img.shape)

In [ ]:
msk = tiff.imread(Train_mask_paths[1])
print(msk.shape)
msk = np.expand_dims(msk, axis=2)
print(msk.shape)

In [ ]:
import re
# Define function to extract number from the file name
def extract_number(filename):
    match = re.search(r'\d+', filename)
    if match:
        return match.group(0)
    return None
# Process each filtered image and its corresponding mask
def process_and_patchify(filtered_image_paths, filtered_mask_paths, output_image_dir, output_mask_dir, patch_size=128):
    for img_path, msk_path in zip(filtered_image_paths, filtered_mask_paths):
        # Read images
        image = tiff.imread(img_path)  # Read each image with tifffile
        mask = tiff.imread(msk_path)  # Load each mask in grayscale
    
        # Transpose image from (bands, height, width) to (height, width, bands)
        if image.ndim == 3:
            image = np.transpose(image, (1, 2, 0))
    
        # Ensure mask has the same number of channels as image
        if mask.ndim == 2:  # If mask is grayscale, expand dimensions to (height, width, 1)
            mask = np.expand_dims(mask, axis=2)
    
        image_name = extract_number(os.path.basename(img_path).split('.')[0])
        mask_name = extract_number(os.path.basename(msk_path).split('.')[0])
    
        print(f"Img name {image_name} and mask name {mask_name}")
        print(f"Before crop Img shape {image.shape} and mask shape {mask.shape}")
    
        SIZE_X = (image.shape[1] // patch_size) * patch_size
        SIZE_Y = (image.shape[0] // patch_size) * patch_size
    
        # Crop image
        image = image[:SIZE_Y, :SIZE_X,:]
    
        # Crop mask
        mask = mask[:SIZE_Y, :SIZE_X,:]
    
        print(f"Post crop Img shape {image.shape} and mask shape {mask.shape}")
    
        # plt.figure(figsize=(12, 8))
        # plt.subplot(121)
        # plt.imshow(image[:,:,0], cmap='gray')  # Display the first band as an example
        # plt.title('Image (Band 1)')
        # plt.subplot(122)
        # plt.imshow(mask[:,:,0], cmap='gray')
        # plt.title('Mask')
        # plt.show()
    
        # Extract patches from each image and mask
        print(f"Now patchifying image {image_name} and mask {mask_name}")
        patches_img = patchify(image, (patch_size, patch_size, image.shape[2]), step=patch_size)  # Assuming the first dimension is bands
        patches_msk = patchify(mask, (patch_size, patch_size, mask.shape[2]), step=patch_size)  # Assuming the first dimension is bands
    #     print(f"Img shape {patches_img.shape} and mask shape {patches_msk.shape}")
    
        useless = 0
        for i in range(patches_img.shape[0]):#
            for j in range(patches_img.shape[1]):#
                single_patch_img = patches_img[i, j,:,:,:]
                single_patch_mask = patches_msk[i, j,:,:,:]
    #             print(f"Img shape {single_patch_img.shape} and mask shape {single_patch_mask.shape}")
    
                # Calculate ratio of non-zero pixels
                temp_mask = single_patch_mask
                temp_mask[np.isnan(temp_mask)] = 0
                val, counts = np.unique(temp_mask, return_counts=True)
                zero_count = np.count_nonzero(temp_mask)
                total_size = temp_mask.size
                nonzero_ratio = (zero_count/total_size) #(counts[0] / counts.sum()) if counts.sum() > 0 else 0
    
    #             print(f'nonzero ration:{nonzero_ratio}, non zero {zero_count} and size {total_size}')
    
                if nonzero_ratio > 0.20:
                    # Save image patch
                    img_filename = f"img_{image_name}_{i}_{j}.tif"
                    tiff.imwrite(os.path.join(output_image_dir, img_filename), single_patch_img)
    
                    # Save mask patch
                    msk_filename = f"msk_{mask_name}_{i}_{j}.tif"
                    tiff.imwrite(os.path.join(output_mask_dir, msk_filename), single_patch_mask)
    
                else:
                    useless += 1
    
        print(f"Number of useless patches: {useless}")

In [ ]:
out_img_train = os.path.join(root_directory, "128_patches_NN/train/images")
out_mask_train = os.path.join(root_directory, "128_patches_NN/train/masks")
out_img_val = os.path.join(root_directory, "128_patches_NN/val/images")
out_mask_val = os.path.join(root_directory, "128_patches_NN/val/masks")
out_img_test = os.path.join(root_directory, "128_patches_NN/test/images")
out_mask_test = os.path.join(root_directory, "128_patches_NN/test/masks")

In [ ]:
process_and_patchify(Train_image_paths, Train_mask_paths, out_img_train, out_mask_train, patch_size=128)

In [ ]:
process_and_patchify(Val_image_paths, Val_mask_paths, out_img_val, out_mask_val, patch_size=128)

In [ ]:
process_and_patchify(Test_image_paths, Test_mask_paths, out_img_test, out_mask_test, patch_size=128)

In [ ]:
# Define function to extract unique identifier from the file name
def extract_identifier(filename):
    match = re.search(r'\d+_\d+_\d+', filename)
    if match:
        return match.group(0)
    return None

In [ ]:
# Visualize random image and mask pair from patches
train_img_dir = out_img_train
train_mask_dir = out_mask_train

# Get list of image and mask files
img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)

# Filter out files without a valid identifier
img_list = [f for f in img_list if extract_identifier(f) is not None]
msk_list = [f for f in msk_list if extract_identifier(f) is not None]

# Sort image and mask lists based on the extracted identifier from filenames
img_list = sorted(img_list, key=lambda x: extract_identifier(x))
msk_list = sorted(msk_list, key=lambda x: extract_identifier(x))

# Ensure the lists have the same number of elements
num_images = len(img_list)
num_masks = len(msk_list)
if num_images != num_masks:
    raise ValueError(f"Number of images ({num_images}) and masks ({num_masks}) do not match!")

# Visualize a random image and mask pair
img_num = random.randint(0, num_images - 1)

img_for_plot = tiff.imread(os.path.join(train_img_dir, img_list[img_num]))
mask_for_plot = tiff.imread(os.path.join(train_mask_dir, msk_list[img_num]))

image_name = extract_identifier(os.path.basename(img_list[img_num]))
mask_name = extract_identifier(os.path.basename(msk_list[img_num]))

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot[0,:, :, 3])  # Display the first band as an example
plt.title('Image (Band EVI)')
plt.subplot(122)
plt.imshow(mask_for_plot[0,:, :, 0], cmap='gray', vmin=0, vmax=100)
plt.title('Mask')
plt.show()

print(f"Selected image: {img_list[img_num]}")
print(f"Selected mask: {msk_list[img_num]}")
print(f"Image identifier: {image_name}")
print(f"Mask identifier: {mask_name}")

In [ ]:
mask_for_plot.shape

In [ ]:
print (len(img_list),len(msk_list))

In [ ]:
# Get list of image and mask files
img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)

# Filter out files without a valid identifier
img_list = [f for f in img_list if extract_identifier(f) is not None]
msk_list = [f for f in msk_list if extract_identifier(f) is not None]
num_images = len(img_list)
img_num = random.randint(0, num_images - 1)

img_for_plot = tiff.imread(os.path.join(train_img_dir, img_list[img_num]))
mask_for_plot = tiff.imread(os.path.join(train_mask_dir, msk_list[img_num]))

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot[0,:,:,2])  # Display the first band as an example
plt.title('Image (Band 1)')
plt.subplot(122)
plt.imshow(mask_for_plot[0,:,:,0])
plt.title('Mask')
plt.show()